In [19]:
import numpy as np
import pysocialforce as psf
import random

Parameters:

In [20]:
# Define initial state of agents
n = 10

# Define goal position
goal_position = np.array([2, 17.5])
goal_position2 = np.array([-10, 7.5])
goal_position3 = np.array([10, 7.5])

goals = np.array([goal_position, goal_position2, goal_position3])
obstacles = [[-10, 0, 15, 15], [2.5, 10, 15, 15], [-5, -5, 7.5, 15], [-5, -5, 5, 0], [5, 5, 7.5, 15], [5, 5, 5, 0], [-5, 5, 0, 0]]

In [21]:
# Helper function: aligns velocity with goal
def initialize_velocity(position, goal):
    direction = goal - position
    norm = np.linalg.norm(direction)
    if norm == 0:
        return np.array([0, 0])
    return direction / norm  # Unit vector towards the goal

# Initialize each agent
initial_state = np.array([])
agent = []
for i in range(n):
	goal = random.choice(goals)
	x = random.uniform(-5, 5)
	y = random.uniform(0, 15)
	vx = initialize_velocity(np.array([x, y]), goal)[0]
	vy = initialize_velocity(np.array([x, y]), goal)[1]
	agent.append([x, y, vx, vy, goal[0], goal[1]])

initial_state = np.array(agent)

# Initialize the simulator
sim = psf.Simulator(
	initial_state, groups=None, obstacles=obstacles, config_file="loes.toml"
)

# Counter for agents that have reached the goal
agents_reached_goal = 0
agents_already_counted = set()  # Set to track counted agents to avoid double counting since removing agents from the simulator is not yet working

# Function to check if an agent has reached the goal
def check_reached_goal(agent_position, y_goal=15, x_goal=5, x_goal2=-5):
	# Check if the agent has reached the goal
	if agent_position[1] > y_goal or agent_position[0] > x_goal or agent_position[0] < x_goal2:
		return True
		

# Perform simulation steps and remove agents that reach the goal 
for step in range(50):
	sim.step(n=1)
	ped_states, group_states = sim.get_states()  # Access the current state of agents

	# Extract the relevant frame (assuming we need the latest frame, index 0 for instance)
	current_state = ped_states[-1]  # Use the last frame for the current state

	new_ped_states = []
	for i, agent_state in enumerate(current_state):
		agent_position = agent_state[:2]  # Ensure agent_position is an array of shape (2,)

		if check_reached_goal(agent_position):
			if i not in agents_already_counted:
				agents_reached_goal += 1
				agents_already_counted.add(i)
		else:
			new_ped_states.append(agent_state)
	
	# Update the simulator's state directly with agents that haven't reached the goal
	sim.ped_states = [np.array(new_ped_states)] if new_ped_states else []

	if len(new_ped_states) == 0:
		break  # Exit the loop if all agents have reached the goal

print(f"Number of agents that reached the goal: {agents_reached_goal}")

Number of agents that reached the goal: 10


In [22]:
# Visualize the scene
with psf.plot.SceneVisualizer(sim, "mult_goals") as sv:
    sv.animate()

INFO:[plot.py:131            __enter__() ] Start plotting.
INFO:[plot.py:162             __exit__() ] Plotting ends.
INFO:[plot.py:166             __exit__() ] Saving animation as mult_goals.gif
DEBUG:[animation.py:651          isAvailable() ] ImageMagick unavailable due to: [Errno 2] No such file or directory: 'convert'
INFO:[animation.py:1050                 save() ] Animation.save using <class 'matplotlib.animation.PillowWriter'>
